### Bank

we build a classifier which can determine whether they will leave or
not using Neural networks

### Load tensorflow

In [0]:
import tensorflow as tf
tf.set_random_seed(42)

In [24]:
tf.__version__

'1.13.0-rc0'

### Collect Data

In [0]:
import keras

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/AI_ML/Resiency6 Lab/bank.csv')

In [28]:
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


**Drop the columns which are unique for all users like IDs**

In [0]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data.Gender=le.fit_transform(data.Gender)

In [0]:
data.Geography=le.fit_transform(data.Geography)

In [0]:
Gender=pd.get_dummies(data.Gender,drop_first=False)
Geography=pd.get_dummies(data.Geography,drop_first=False)

In [0]:
data.drop(['Gender','Geography'],axis=1,inplace=True)

In [0]:
data=pd.concat([data,Gender,Geography],axis=1)

In [35]:
data.head(5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,0,1,0,1,2
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0,0,0,1


**Divide the data set into Train and test sets and Distinguish the feature and target set**

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
trainX,testX= train_test_split(data,test_size=0.3,random_state=100)

In [0]:
trainY=trainX.pop("Exited")
testY=testX.pop("Exited")

In [0]:
trainY = tf.keras.utils.to_categorical(trainY,num_classes=2)
testY = tf.keras.utils.to_categorical(testY, num_classes=2)

**normalse data, Initialize and build the model**

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=(13,)))

#Add First layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(2, activation='softmax'))

#Defining the optimizer

#sgd_optimizer = tf.keras.optimizers.SGD(lr=0.01,clipnorm=1.)

adagard=tf.keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer=adagard, loss='binary_crossentropy',metrics=['accuracy'])

### Execute the model using model.fit()

In [52]:
model.fit(trainX.values, trainY, 
          validation_data=(testX.values, testY), 
          epochs=30,          batch_size=32)

Train on 7000 samples, validate on 3000 samples
Epoch 1/30
7000/7000 [==============================] - 1s 153us/sample - loss: 0.3724 - acc: 0.8427 - val_loss: 0.3646 - val_acc: 0.8483
Epoch 2/30
7000/7000 [==============================] - 1s 154us/sample - loss: 0.3708 - acc: 0.8409 - val_loss: 0.3644 - val_acc: 0.8477
Epoch 3/30
7000/7000 [==============================] - 1s 163us/sample - loss: 0.3690 - acc: 0.8434 - val_loss: 0.3639 - val_acc: 0.8490
Epoch 4/30
7000/7000 [==============================] - 1s 163us/sample - loss: 0.3704 - acc: 0.8477 - val_loss: 0.3638 - val_acc: 0.8477
Epoch 5/30
7000/7000 [==============================] - 1s 164us/sample - loss: 0.3697 - acc: 0.8459 - val_loss: 0.3633 - val_acc: 0.8493
Epoch 6/30
7000/7000 [==============================] - 1s 162us/sample - loss: 0.3680 - acc: 0.8467 - val_loss: 0.3632 - val_acc: 0.8497
Epoch 7/30
7000/7000 [==============================] - 1s 163us/sample - loss: 0.3694 - acc: 0.8449 - val_loss: 0.3630 - va

In [0]:
predictions=model.predict(testX)

In [62]:
predictions

array([[0.83925486, 0.16074514],
       [0.9209078 , 0.07909223],
       [0.92580724, 0.07419279],
       ...,
       [0.88829494, 0.11170504],
       [0.9726927 , 0.02730733],
       [0.63105625, 0.36894378]], dtype=float32)

In [0]:
import numpy as np
final_predictions=[]
for x in range(len(predictions)):
  if(np.argmax(predictions[x])>=0.5):
    final_predictions.append(1)
  else:
    final_predictions.append(0)
    

In [65]:
print("Firts 10 predictions : ")
for i in range(20):
  print("predicted value: ",final_predictions[i], " actual value: ",testy.iloc[i])

Firts 10 predictions : 
predicted value:  0  actual value:  1
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  1
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  1
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  1
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0
predicted value:  0  actual value:  0


## Review model

In [66]:
 model.evaluate(testX,testY)

3000/3000 [==============================] - 0s 69us/sample - loss: 0.3582 - acc: 0.8550


[0.3582172784805298, 0.855]

In [82]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(testy, final_predictions)

array([[2309,   67],
       [ 368,  256]])